<h1 align="center"><font size="5">Applied Data Science Capstone - Nikhil M</font></h1>

This notebook contains the work for the Applied Data Science Capstone.

Loading the required libraries:

In [17]:
import numpy as np
import pandas as pd

In [18]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [19]:
import requests # library to handle requests
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from lxml import etree

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [20]:
#Initialize constants for FourSquare
CLIENT_ID = '5SVQZ3G4RIN54PVDABFRLPGLCS0Y0ZF5IAZPP4NXQEWNHYA4' # your Foursquare ID
CLIENT_SECRET = 'ZAIO0JVOYHPVMYX011CCTQ2BFHZSEHT3THLJGEXPVFLSHN40' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [21]:
#Extract neighborhoods in Toronto
base_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(base_url) 

#Extract table with list of neighborhoods
df_neighborhoods = tables[0]

#Assign column headers & drop unnecessary columns
df_neighborhoods.columns = df_neighborhoods.iloc[0]
df_neighborhoods = df_neighborhoods.reindex(df_neighborhoods.index.drop(0))
df_neighborhoods = df_neighborhoods[df_neighborhoods["Borough"] != "Not assigned"]
df_neighborhoods = df_neighborhoods.reset_index(drop=True)

#df_neighborhoods.head()

group_by_neighborhoods = df_neighborhoods.groupby(['Postcode'])

column_headers = ['Postcode', 'Borough', 'Neighbourhood']
final_neighborhood_data = pd.DataFrame(columns=column_headers)
final_neighborhood_data.columns = column_headers

for name, group in group_by_neighborhoods:
    #Get individual neighborhood names
    combinedNames = ""
    for colName in group["Neighbourhood"]:
        combinedNames = colName + ',' + combinedNames
    combinedNames = combinedNames[:-1]    

    #Create a row and add it to the new dataframe as required in the question
    finalDataRow = pd.Series([name, group["Borough"].values[0], combinedNames], index=column_headers)    
    final_neighborhood_data = final_neighborhood_data.append(finalDataRow, ignore_index=True)
    
final_neighborhood_data.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"West Hill,Morningside,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
#CSV for lats and longs for Toronto
coordinates_url = "https://cocl.us/Geospatial_data"
coordinates_data = pd.read_csv(coordinates_url) 
coordinates_data.columns = ["Postcode", "Latitude", "Longitude"]

merged_neighborhood_data = pd.merge(final_neighborhood_data,
                                   coordinates_data,
                                   on="Postcode")
merged_neighborhood_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [117]:
geolocator = Nominatim(user_agent="toronto_explorer")
toronto_location = geolocator.geocode("Toronto")

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_location.latitude, toronto_location.longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_neighborhood_data['Latitude'], 
                                           merged_neighborhood_data['Longitude'], 
                                           merged_neighborhood_data['Borough'], 
                                           merged_neighborhood_data['Neighbourhood']):
    
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    return(nearby_venues)

toronto_neighborhood_venues = getNearbyVenues(names=merged_neighborhood_data['Neighbourhood'],
                                   latitudes=merged_neighborhood_data['Latitude'],
                                   longitudes=merged_neighborhood_data['Longitude'])

toronto_neighborhood_venues.head()

KeyError: 'groups'

In [70]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_neighborhood_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()


In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [100]:
#dummy line
neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop('Cluster Labels', 1)


In [101]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = df_neighborhoods
toronto_merged = merged_neighborhood_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

#Clean up cluster label values to display on map
toronto_merged_final = toronto_merged.drop(toronto_merged[toronto_merged['Cluster Labels'].isna()].index)
toronto_merged_final['Cluster Labels'] = toronto_merged_final['Cluster Labels'].astype(int)

In [116]:
geolocator = Nominatim(user_agent="toronto_explorer")
toronto_location = geolocator.geocode("Toronto")

# create map
map_clusters = folium.Map(location=[toronto_location.latitude, toronto_location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_final['Latitude'], 
                                  toronto_merged_final['Longitude'], 
                                  toronto_merged_final['Neighbourhood'], 
                                  toronto_merged_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters